In [29]:
# !pip install smolagents
# !pip install prettyprinter gradio-client --quiet
# !pip install arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents --quiet

In [2]:
from prettyprinter import pprint

## Setup Model and Telemetry

### Model: 4o-mini

In [3]:
import os
from smolagents import CodeAgent, LiteLLMModel
from smolagents import OpenAIServerModel

model = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

### Telemetry

This is useful for tracking the inputs and outputs, since tracking usage of agents is complicated we use OpenTelemetry.

Run

```bash
python -m phoenix.server.main serve
```
and check at  http://0.0.0.0:6006/projects/  to see your usage

In [30]:
# from opentelemetry import trace
# from opentelemetry.sdk.trace import TracerProvider
# from opentelemetry.sdk.trace.export import BatchSpanProcessor

# from openinference.instrumentation.smolagents import SmolagentsInstrumentor
# from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

# endpoint = "http://0.0.0.0:6006/v1/traces"
# trace_provider = TracerProvider()
# trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

# SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

## Basic Query without any Tools

In [9]:
agent = CodeAgent(tools=[], model=model, add_base_tools=False)

In [10]:
agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import math                                                                                                      
                                                                                                                   
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fibonacci_118 = fibonacci(118)                                                                                   
  final_answer(fibonacci_118)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 0: Duration 3.12 seconds| Input tokens: 1,961 | Output tokens: 129]

2046711111473984623691759

In [12]:
agent.run(
    "What is the capital of france?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of france?                                                                                  │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Paris")                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Paris

[Step 0: Duration 2.16 seconds| Input tokens: 1,954 | Output tokens: 37]

'Paris'

## Basic Query with Base Tools

Base tools: DuckDuckGo search tool, visit webpage tool, python interpreter tool, and audio transcription tool

In [5]:
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

In [6]:
agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_118 = fibonacci(118)                                                                                         
  print(fib_118)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
2046711111473984623691759

Out: None

[Step 0: Duration 6.01 seconds| Input tokens: 2,084 | Output tokens: 114]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(2046711111473984623691759)                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 1: Duration 2.30 seconds| Input tokens: 4,411 | Output tokens: 175]

2046711111473984623691759

Since its a code agent, the agent first creates code to solve the given problem, the code is executed to give output and the LLM again verifies the answer to give the final answer.

In [7]:
agent.run(
    "What is the capital of france?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of france?                                                                                  │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  capital_of_france = web_search(query="What is the capital of France?")                                           
  print(capital_of_france)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Paris - Wikipedia](https://en.wikipedia.org/wiki/Paris)
Paris is a global centre of finance, diplomacy, culture, and gastronomy, with an estimated population of 2.1 
million in 2023. It has many famous landmarks, museums, and historical districts, and is home to several 
international organizations and sports clubs.

[Paris | Definition, Map, Population, Facts, & History | Britannica](https://www.britannica.com/place/Paris)
Paris, city and capital of France, located along the Seine River, in the north-central part of the country. Paris 
is one of the world's most important and attractive cities, famed for its gastronomy, haute couture, painting, 
literature, and intellectual community. Learn more about Paris in this article.

[What is the Capital of France? - 
WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-france.html)
Learn about Paris, the largest and most populous city in France, and its history, geography, economy, tourism, and 
administration. Find out why Paris is called the City of Light and the City of Love.

[France | History, Maps, Flag, Population, Cities, Capital, & Facts ...](https://www.britannica.com/place/France)
France, a country of northwestern Europe, is historically and culturally among the most important countries in the 
Western world. It has also played a highly significant role in international affairs for centuries. Its capital is 
Paris, one of the most important cultural and commercial centers in the world.

[Paris - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Paris)
Paris is the capital city of France and the largest city in France. It has a rich history, many art museums, 
historical buildings, and a famous landmark, the Eiffel Tower.

[List of capitals of France - Wikipedia](https://en.wikipedia.org/wiki/List_of_capitals_of_France)
The capital of France has been Paris since its liberation in 1944. Learn about the historical and provisional seats
of the government of France from 486 to 1944.

[Paris: Facts & Related Content - Encyclopedia Britannica](https://www.britannica.com/facts/Paris)
Paris is the capital of France, located in the north-central part of the country. It is a major centre of commerce,
culture, and tourism, with famous landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame de Paris.

[Capital of France - Simple English Wikipedia, the free 
encyclopedia](https://simple.wikipedia.org/wiki/Capital_of_France)
Learn about the history and current status of the capital of France, which is Paris. Find out which other cities 
have been the national capital in different periods of time.

[What is the Capital of France? - GeeksforGeeks](https://www.geeksforgeeks.org/what-is-the-capital-of-france/)
Learn that the capital of France is Paris, a global center for art, fashion, culture, and history. Find out more 
about Paris' landmarks, history, districts, and time difference with the United States.

[What is the Capital of France? - Mappr](https://www.mappr.co/capital-cities/france/)
Learn why Paris is the capital of France and how it became a global city with a rich cultural heritage. Discover 
its geography, climate, population, landmarks, and industries.

Out: None

[Step 0: Duration 4.02 seconds| Input tokens: 2,077 | Output tokens: 63]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Paris")                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Paris

[Step 1: Duration 2.26 seconds| Input tokens: 5,001 | Output tokens: 108]

'Paris'

In [4]:
# help(agent)

## Examining the Messages internal to Agent

In [8]:
len(agent.memory.steps)

3

In [9]:
agent.memory.steps[1].__match_args__

('model_input_messages',
 'tool_calls',
 'start_time',
 'end_time',
 'step_number',
 'error',
 'duration',
 'model_output_message',
 'model_output',
 'observations',
 'observations_images',
 'action_output')

In [14]:
# pprint(agent.memory.steps[-1].model_input_messages)

In [13]:
pprint(agent.write_memory_to_messages())

[
    {
        'role': smolagents.models.MessageRole(''),
        'content': [
            {
                'type': 'text',
                'text':
                    "You are an expert assistant who can solve any task using "
                    "code blobs. You will be given a task to solve as best you "
                    "can.\nTo do so, you have been given access to a list of "
                    "tools: these tools are basically Python functions which you "
                    "can call with code.\nTo solve the task, you must plan "
                    "forward to proceed in a series of steps, in a cycle of "
                    "'Thought:', 'Code:', and 'Observation:' sequences.\n\nAt "
                    "each step, in the 'Thought:' sequence, you should first "
                    "explain your reasoning towards solving the task and the "
                    "tools that you want to use.\nThen in the 'Code:' sequence, "
                    "you should write the code in si

## Tool Calling Agent

Instead of code uses JSON structured output

In [52]:
from smolagents import ToolCallingAgent

agent = ToolCallingAgent(tools=[], model=model, add_base_tools=True)
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog'?                                    │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'visit_webpage' with arguments: {'url': 'https://huggingface.co/blog'}                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Hugging Face – Blog

|!|Hugging Face's logo](/front/assets/huggingface_logo-noborder.svg)
Hugging Face](/)

* |Models](/models)
* |Datasets](/datasets)
* |Spaces](/spaces)
* |Posts](/posts)
* |Docs](/docs)
* |Enterprise](/enterprise)
* |Pricing](/pricing)
* ---
* |Log In](/login)
* |Sign Up](/join)

Blog, Articles, and discussions
===============================

New Article

|Everything](/blog)
|community](/blog?tag=community)|guide](/blog?tag=guide)|open source 
collab](/blog?tag=open-source-collab)|partnerships](/blog?tag=partnerships)|research](/blog?tag=research)|NLP](/blo
g?tag=nlp)|Audio](/blog?tag=audio)|CV](/blog?tag=cv)|RL](/blog?tag=rl)|ethics](/blog?tag=ethics)|Diffusion](/blog?t
ag=diffusion)|Game Development](/blog?tag=game-dev)|RLHF](/blog?tag=rlhf)|Leaderboard](/blog?tag=leaderboard)|Case 
Studies](/blog?tag=case-studies)

|!|](/blog/assets/ai_art_newsletter_1/thumbnail.png)

The AI tools for Art Newsletter - Issue 1
-----------------------------------------

By 
|linoyts](/linoyts)

January 31, 2025
•

29](/blog/ai-art-newsletter-jan-25)
Community Articles
|view all](/blog/community)
|#### o3-mini vs Deepseek-R1

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/65bb837dbfb878f46c77de4c/UVtVbF_3rdt0DC8xTkpL1.jpeg)
prithivMLmods](/prithivMLmods)
• 
about 2 hours ago
•

7](/blog/prithivMLmods/o3-mini-vs-deepseek-r1)
|#### Activation Steering: A New Frontier in AI Control—But Does It Scale?

By
|!|](/avatars/494015182ac0d9f56eecbd5764eb4805.svg)
royswastik](/royswastik)
• 
about 8 hours ago](/blog/royswastik/activation-steering)
|#### Open-R1: Update #1

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/651e96991b97c9f33d26bde6/QG_uzb2VV3OeKO0c61ynh.png)
open-r1](/open-r1)
• 
about 11 hours ago
•

26](/blog/open-r1/update-1)
|#### The AHA Indicator

By
|!|](/avatars/cf21cf2c8f1c9d5a8fb35761acdef04b.svg)
etemiz](/etemiz)
• 
about 18 hours ago
•

1](/blog/etemiz/aha-indicator)
|#### LLM Dataset Formats 101: A No‐BS Guide for Hugging Face Devs

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/6702d5463fe7cea28adbc921/fbchFJFH9oGyJuPhJ-E-Q.png)
tegridydev](/tegridydev)
• 
1 day ago
•

3](/blog/tegridydev/llm-dataset-formats-101-hugging-face)
|#### Why we (don't) need export control

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/65e330e7edc2f7306e252448/oYAOGhbPaXDTbEoJoSLMB.jpeg)
as-cle-bert](/as-cle-bert)
• 
1 day ago
•

7](/blog/as-cle-bert/why-we-dont-need-export-control)
|#### AI Agents for Trip Planning: Automating Itinerary Generation with KaibanJS

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/19rKxRAjkXIGrtUg0iwFz.jpeg)
darielnoel](/darielnoel)
• 
2 days ago](/blog/darielnoel/ai-agents-trip-planning-kaibanjs)
|#### Replicating DeepSeek R1 for Information Extraction

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/1658166666371-noauth.png)
Ihor](/Ihor)
• 
2 days ago
•

14](/blog/Ihor/replicating-deepseek-r1-for-information-extraction)
|#### Enhancing AI Agents with the TextFile RAG Search Tool in KaibanJS

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/19rKxRAjkXIGrtUg0iwFz.jpeg)
darielnoel](/darielnoel)
• 
2 days ago](/blog/darielnoel/enhancing-ai-agents-textfile-rag-kaibanjs)
|#### Mini-R1: Reproduce Deepseek R1 „aha moment“ a RL tutorial

By
|!|](https://cdn-avatars.huggingface.co/v1/production/uploads/651e96991b97c9f33d26bde6/QG_uzb2VV3OeKO0c61ynh.png)
open-r1](/open-r1)
• 
2 days ago
•

21](/blog/open-r1/mini-r1-contdown-game)
|#### The Power of Open-Source AI

By
|!|](/avatars/b70e38c2ecb3c7d27feebbd7b3df8ee8.svg)
mediiiiii3](/mediiiiii3)
• 
2 days ago](/blog/mediiiiii3/../../blog/mediiiiii/intro-to-ai-part1)
|#### The Power of Open-Source AI

By
|!|](/avatars/b70e38c2ecb3c7d27feebbd7b3df8ee8.svg)
mediiiiii3](/mediiiiii3)
• 
2 days ago](/blog/mediiiiii3/../../mediiiiii/intro-to-ai-part1)
|#### Introduction to Artificial Intelligence

By
|!|](/avatars/fd1e6d385

[Step 0: Duration 3.76 seconds| Input tokens: 1,270 | Output tokens: 21]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Hugging Face – Blog'}                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Hugging Face – Blog

[Step 1: Duration 1.80 seconds| Input tokens: 6,337 | Output tokens: 40]

'Hugging Face – Blog'

In [55]:
pprint(agent.memory.steps[1].model_input_messages)

[
    {
        'role': smolagents.models.MessageRole(''),
        'content': [
            {
                'type': 'text',
                'text':
                    'You are an expert assistant who can solve any task using  '
                    'tool calls. You will be given a task to solve as best you '
                    'can.\nTo do so, you have been given access to the following '
                    'tools: \'python_interpreter\', \'web_search\', '
                    '\'visit_webpage\', \'final_answer\'\n\nThe tool call you '
                    'write is an action: after the tool is executed, you will get '
                    'the result of the tool call as an "observation".\nThis '
                    'Action/Observation can repeat N times, you should take '
                    'several steps when needed.\n\nYou can use the result of the '
                    'previous action as input for the next action.\nThe '
                    'observation will always be a string: 

In [18]:
# print(agent.system_prompt_template)

## Planning Steps

In [25]:
agent = CodeAgent(tools=[], model=model, add_base_tools=True, planning_interval=3)

In [23]:
result = agent.run(
    "How long would a cheetah at full speed take to run the length of Pont Alexandre III?",
)
result

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How long would a cheetah at full speed take to run the length of Pont Alexandre III?                            │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pont_length_search = web_search(query="Pont Alexandre III length")                                               
  print("Pont Alexandre III length:", pont_length_search)                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Pont Alexandre III length: ## Search Results

[Pont Alexandre III - Wikipedia](https://en.wikipedia.org/wiki/Pont_Alexandre_III)
The Pont Alexandre III (French pronunciation: [pɔ̃ alɛksɑ̃dʁ tʁwa]) is a deck arch bridge that spans the Seine in 
Paris. It connects the Champs-Élysées quarter with those of the Invalides and Eiffel Tower.

[The Pont Alexandre III is the most beautiful bridge in 
Paris.](https://monsieur-de-france.com/en/bridge-pont-alexandre-iii-paris-france)
The Alexandre III Bridge in figures The Alexandre III Bridge at sunrise / Photo chosen by Monsieurdefrance.com: 
Manjik via dépositphotos. The bridge is 152 meters long in total, but the arch is 107 meters long and 45 meters 
wide. It is made of cast steel and rests on two enormous "abutments" that withstand the pressure exerted by the 
arch.

[10 Fun Facts About The Pont Alexandre III](https://art-facts.com/facts-about-the-pont-alexandre-iii/)
1. The bridge spans the River Seine The Pont Alexandre III ("pont" means "bridge in French) spans the famous River 
Seine in Paris. It's one of the most famous bridges in the city and is located in a very important area. It 
connects the neighborhood of the Champs-Élysées, the most famous avenue in the city, and the neighborhood where we 
can find two other famous landmarks, Les ...

[17 Famous Bridges in France: Architectural Time Travel - Salut from 
Paris](https://www.salutfromparis.com/famous-bridges-in-france/)
Its slender design offers minimal obstruction to the view, making a journey across it an unforgettable experience. 
The Millau Viaduct not only symbolizes architectural and engineering prowess but also human ingenuity, representing
the pinnacle of famous bridges in France. #2 Pont Alexandre III, Paris (built in 1900, 160m length)

[Pont Alexandre III, Paris, France - 
LatLong](https://www.latlong.net/place/pont-alexandre-iii-paris-france-32915.html)
Pont Alexandre III, Paris, France Lat Long Coordinates Info The latitude of Pont Alexandre III, Paris, France is 
48.863869, and the longitude is 2.313427. Pont Alexandre III, Paris, France is located at France country in the 
Bridges place category with the gps coordinates of 48° 51' 49.9284'' N and 2° 18' 48.3372'' E.

[Pont Alexandre III - Come to 
Paris](https://www.cometoparis.com/paris-guide/paris-monuments/pont-alexandre-iii-s951)
The inscription reads 'On April 14th 1900, Emile Loubet, President of the French Republic, opened the Exposition 
Universelle and inaugurated the Pont Alexandre III'. At 154 metres in length and 45 metres in width, this enormous 
bridge connects the Esplanade des Invalides to the Avenue Wilson Churchill.

[Pont Alexandre III's lavish decoration - Travel France 
Online](https://www.travelfranceonline.com/pont-alexandre-iiis-lavish-decoration/)
The Pont Alexandre III is the widest and lowest bridge in Paris; it indeed measures 154 meters in length and 40 
meters in width. Pont Alexandre III, a masterpiece of metal architecture

[Pont Alexandre III - Seine](https://seine.com/bridges/pont-alexandre-iii.php)
The Pont Alexandre III is known for its stunning design, which features ornate Art Nouveau decorations, sculptures,
and lamp posts. The bridge is 160 meters long and has a single span that is supported by two massive pillars 
decorated with golden statues of Pegasus, the winged horse of Greek mythology.

[Pont Alexandre III - Paris](https://www.paris.si/pont-alexandre-iii/)
The Pont Alexandre III ranks as the most exuberant bridge in the whole of Paris. With its many lampshade 
decorations, the bridge lights up at night, providing a captivating sight when darkness falls.

[Pont Alexandre Iii (EN) - TravelMate - 
MyWoWo](https://travelmate.tech/en/france/paris/invalides/building/pont-alexandre-iii)
The Alexander III Bridge is one of the most famous bridges in Paris. It was built between 1896 and 1900 and spans 
the Seine River, connecting the Esplanade des Invalides with the Avenue des Champs-Élysées. It is known fo

[Step 0: Duration 7.88 seconds| Input tokens: 2,090 | Output tokens: 118]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pont_length_meters = 154  # Length of Pont Alexandre III in meters                                               
  cheetah_speed_mps = 75 * 0.44704  # Convert 75 mph to meters per second                                          
  time_taken_seconds = pont_length_meters / cheetah_speed_mps                                                      
  final_answer(time_taken_seconds)  # Display the final answer                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 4.593175853018373

[Step 3: Duration 11.40 seconds| Input tokens: 18,052 | Output tokens: 694]

4.593175853018373

In [26]:
result = agent.run(
    "How long would it take for a great indian bustard to run the full length of the golden bridge in sanfranciso?",
)
result

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How long would it take for a great indian bustard to run the full length of the golden bridge in sanfranciso?   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  bridge_length = web_search(query="length of the Golden Gate Bridge")                                             
  bustard_speed = web_search(query="average running speed of great Indian bustard")                                
  print("Bridge length:", bridge_length)                                                                           
  print("Bustard speed:", bustard_speed)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Bridge length: ## Search Results

[Golden Gate Bridge - Wikipedia](https://en.wikipedia.org/wiki/Golden_Gate_Bridge)
The Golden Gate Bridge is a suspension bridge spanning the Golden Gate, the one-mile-wide (1.6 km) ... The total 
length of the Golden Gate Bridge from abutment to abutment is 8,981 feet (2,737 m). [56] The Golden Gate Bridge's 
clearance above high water averages 220 feet ...

[Golden Gate Bridge ‑ Length, Facts & Height | 
HISTORY](https://www.history.com/topics/landmarks/golden-gate-bridge)
The Golden Gate Bridge, opened in 1937, is an iconic suspension bridge connecting the city of San Francisco to 
Marin County, California. It spans almost two miles across the Golden Gate, the ...

[Design & Construction Stats - Statistics & Data | Golden 
Gate](https://www.goldengate.org/bridge/history-research/statistics-data/design-construction-stats/)
Total length of Bridge including approaches from abutment to abutment is 1.7 miles (8,981 ft or 2,737 m). ... The 
Golden Gate Bridge has 250 pairs of vertical suspender ropes that are spaced 50 feet apart across both sides of the
Bridge. Each suspender rope is 2-11/16 inches in diameter. All of the ropes were replaced between 1972 and 1976 ...

[Golden Gate Bridge | History, Construction, & Facts | 
Britannica](https://www.britannica.com/topic/Golden-Gate-Bridge)
Golden Gate Bridge, suspension bridge spanning the Golden Gate in California to link San Francisco with Marin 
county to the north. Upon its completion in 1937, it was the tallest and longest suspension bridge in the world. 
Learn more about the history and construction of the Golden Gate Bridge.

[How Long is the Golden Gate Bridge? - U.S. National Park 
Service](https://www.nps.gov/places/000/how-long-is-the-golden-gate-bridge.htm)
For four years, from January 1933 to April 1937, bridge construction proceeded. Upon opening in May 1937, the 
Golden Gate Bridge was the longest suspension bridge in the world. The middle span between the 44,000-ton towers 
stretches 4,200 feet. Even today, few suspension bridges are as long, and none more spectacular.

[Facts & Figures About the Bridge - Exhibits Area 1 | Golden 
Gate](https://www.goldengate.org/exhibits/facts-and-figures-about-the-bridge/)
Learn about the history, construction, and features of the Golden Gate Bridge, the world's longest suspension 
bridge when it opened in 1937. The main span of the bridge is 4,200 feet (1,280 m) long, and the total length is 
1.7 miles (2,737 m).

[How Big is the Golden Gate Bridge? - Dimensions 
Guide](https://www.dimensionsguide.com/how-big-is-the-golden-gate-bridge/)
The dimensions of the Golden Gate Bridge made it the longest suspension bridge span when it was finished in 1937. 
It is 1.7 mi (2.7 km) or 8,981 ft (2,737.4 m) long. Other bridges have surpassed it in length.

[Golden Gate Bridge Fun Facts - The Presidio (San 
Francisco)](https://presidio.gov/explore/blog/golden-gate-bridge-fun-facts)
How long is the Golden Gate Bridge? When it opened, the Golden Gate Bridge was the longest and tallest suspension 
bridge in the world. The Golden Gate Bridge measures 8,981 ft in total length, which equates to 1.7 miles (or 2.7 
kilometers), making it a true wonder of engineering ingenuity. How high is the Golden Gate Bridge from the ocean?

[The Golden Gate Bridge: Facts, History & Profile - 
Infoplease](https://www.infoplease.com/geography/monuments-landmarks/the-golden-gate-bridge)
Fast Golden Gate Bridge Facts. Length: ... The Golden Gate Bridge is a suspension bridge, which means that the 
roadway is suspended from two main cables that are anchored to massive concrete anchorages on either side of the 
bridge. The bridge is 1.7 miles long and 90 feet wide, making it remain one of the longest suspension bridges in 
the world

[Golden Gate Bridge - 
HistoricBridges.org](https://historicbridges.org/bridges/browser/?bridgebrowser=california/goldengate/)
The Golden Gate Bridge is one of the most well-known and belove

[Step 0: Duration 12.97 seconds| Input tokens: 2,096 | Output tokens: 176]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  bridge_length_feet = 8981  # length of the Golden Gate Bridge in feet                                            
  bustard_speed_mph = 30  # average speed in miles per hour                                                        
  bustard_speed_fph = bustard_speed_mph * 5280  # convert speed to feet per hour                                   
                                                                                                                   
  time_hours = bridge_length_feet / bustard_speed_fph  # time in hours                                             
  time_minutes = time_hours * 60  # convert time to minutes                                                        
  final_answer(time_minutes)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 3.4018939393939394

[Step 2: Duration 17.09 seconds| Input tokens: 11,792 | Output tokens: 384]

3.4018939393939394

## Tools from HF Spaces

In [31]:
from smolagents import Tool, load_tool

In [35]:
image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.


In [37]:
# image_generation_tool("A sunny beach")